In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import tensorflow as tf
import numpy as np
import collections

/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1600px;height:750px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
x0 = np.array([[1.], [1.]])  # 2x1 vector 

A = tf.constant(np.array([[-0.5, 0], [0, -0.6]]), dtype=tf.float32, name='A')  # system dynamics
B = tf.constant(np.array([[1.], [1.]]), dtype=tf.float32, name='B')

Q = tf.constant([[1.,0.], [0.,1.]], name='Q')  # weighting
R = tf.constant(1., name='R')

x = tf.constant(x0, dtype=tf.float32, name="state")

T = 10

In [5]:
####### Defining network #######
# input: state x
# output: control u

input_layer = tf.placeholder(tf.float32, (None,2), name='in_layer')
# fc1 = tf.layers.dense(inputs=input_layer, units=2,) #activation=tf.nn.sigmoid)
# fc2 = tf.layers.dense(inputs=fc1, units=2, activation=tf.nn.sigmoid)
u = tf.layers.dense(inputs=input_layer, units=1, name='u_out_layer', reuse=tf.AUTO_REUSE)

### LOSS FUNCTION ### 
loss = tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), 
              tf.matmul(tf.transpose(u), tf.multiply(R, u)), name='loss')

# xs = tf.identity(x, name='xs')
# us = tf.constant(0, name='us')
xs = x
us = u

# cond = lambda i, x, l, xs, us: i < T

# def body(i, x, l, xs, us):
#     next_i = i+1
#     next_x = tf.add(tf.matmul(A, x), tf.multiply(u,B))
#     next_l = tf.add(l,
#                     tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x),
#                            tf.matmul(tf.transpose(u), tf.multiply(R, u))))
#     next_xs = tf.concat(xs, next_x)
#     next_us = tf.concat(us, u)
#     return (next_i, next_x, next_l, next_xs, next_us)

# i, xloss_f, traj_f = tf.while_loop(cond, body, 
#                                    loop_vars=[tf.constant(0), x, loss, xs, us],
#                                    shape_invariants=[tf.TensorShape([1,]), tf.TensorShape([2, 1]), 
#                                                      tf.TensorShape([1,]) , tf.TensorShape([2, None]), 
#                                                      tf.TensorShape([1, None])])
# train = tf.train.GradientDescentOptimizer(0.01).minimize(xloss_f.loss)

for i in range(T):
    # LQR loss 
#     x_term = tf.matmul(tf.matmul(tf.transpose(x), Q), x, name='x_term')
#     u_term = tf.matmul(tf.transpose(u), tf.multiply(R, u), name='u_term')
#     loss = tf.add(loss, tf.add(x_term, u_term), name='loss')  # accumulate loss
    
    # Dynamics: advancing the system dynamics
    Ax = tf.matmul(A, x, name='Ax'+str(i))
    Bu = tf.multiply(u, B, name='Bu'+str(i))  # tf.multiply because u is a scalar
    x = tf.add(Ax, Bu, name='state'+str(i))  # next state vector

    loss = tf.add(loss, tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), tf.matmul(tf.transpose(u), tf.multiply(R, u))), name='loss'+str(i))  # accumulate loss    
    
    u = tf.layers.dense(inputs=tf.transpose(x), units=1, name='u_out_layer', reuse=True)
    
    xs = tf.concat([xs, x], 1)
    us = tf.concat([us, u], 1)
    
opt = tf.train.GradientDescentOptimizer(0.0001)
train = opt.minimize(loss)
grads_and_vars = opt.compute_gradients(loss)

In [6]:
# show_graph(tf.get_default_graph().as_graph_def())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(250):
#         sess.run(train)
        sess.run(train, feed_dict={input_layer : x0.T})
        if i % 50 == 0:
#             results = sess.run([xs, us, loss, grads_and_vars], feed_dict={input_layer : x0.T})
            results = sess.run([xs, us, loss], feed_dict={input_layer : x0.T})
            labels  = "xs us loss".split(' ')
            print('training iteration', i)
            for label,result in zip(*(labels,results)) :
                print(label)
                print(result)
                print('')
            for g, v in grads_and_vars:
                print('gradients')
                print(str(sess.run(g, feed_dict={input_layer : x0.T})) + " - " + v.name)
            print('----------------------')

training iteration 0
xs
[[ 1.         -0.4970896   0.37587583 -0.32742983  0.29586723 -0.26849094
   0.24479899 -0.22247541  0.2029894  -0.1844219   0.16834217]
 [ 1.         -0.5970896   0.48558483 -0.43084282  0.39065802 -0.35495216
   0.3235248  -0.29419082  0.2682662  -0.24388692  0.22246337]]

us
[[ 0.00291041  0.12733105 -0.13949192  0.13215232 -0.12055733  0.11055351
  -0.10007592  0.0917517  -0.08292719  0.07613122 -0.06870268]]

loss
[[4.407901]]

gradients
[[-15.519047]
 [-18.11099 ]] - u_out_layer/kernel:0
gradients
[-2.9853582] - u_out_layer/bias:0
----------------------
training iteration 50
xs
[[ 1.         -0.3820171   0.281263   -0.22121912  0.20247486 -0.161154
   0.1517719  -0.11763153  0.1143171  -0.0853859   0.0865545 ]
 [ 1.         -0.48201713  0.37946475 -0.3082665   0.2768252  -0.2260117
   0.20680192 -0.16582674  0.15499738 -0.12122578  0.11659703]]

us
[[ 0.11798289  0.09025445 -0.08058763  0.09186529 -0.05991656  0.07119491
  -0.04174558  0.05550133 -0.028227

## compute optimal loss with true LQR ricatti equation formulation

In [7]:
import controlpy

In [30]:
if type(A) != np.ndarray:  # if tensors not already evaluated
    with tf.Session() as sess:
        A, B, Q, R = sess.run([A, B, Q, R])

K, P, eig = controlpy.synthesis.controller_lqr_discrete_time(A, B, Q, R)
x = x0
u = (-K@x)
xs, us = np.array(x), np.array(u)
loss = 0
for i in range(T):
    loss += x.T@Q@x + u.T*R*u
    x = A@x + B@u
    u = (-K@x)
    xs = np.hstack([xs, x])
    us = np.hstack([us, u])

In [31]:
us.shape

(1, 11)

In [33]:
us

array([[ 3.80738770e-01, -6.70907865e-02,  1.27556619e-02,
        -2.91909818e-03,  9.10254339e-04, -3.82533515e-04,
         1.90269282e-04, -1.01185839e-04,  5.50383161e-05,
        -3.01523452e-05,  1.65556325e-05]])